# Analysis of NASA Apache Web Logs and feature recommendation for website optimization

## Problem statement:
#### Website optimization to predict traffic load; so that company can manage resources for handling peak traffic load

#### Segment visitors according to how they engage with the website, map these segments to known customers in order to feed them into the right channels

---
*  Author - Vikram H Patil
*  Date: 29/11/2019
*  Environment: Python 3.6.1 and Jupyter notebook


----
## Table of contents

### 1 - Data gathering and reading in Python
### 2 - Data wrangling
### 3 - EDA performed in Tableau
### 4 - Creating new Features
### 5 - Conclusison and recommendation



----

### 1- Data gathering and reading in Python

In [1]:
# Importing all necessary Libraries

import warnings
from datetime import datetime, timedelta
import swifter
warnings.filterwarnings("ignore")
import pandas as pd
import csv


import socket
from geolite2 import geolite2
import numpy as np

#### Reading the two .tsv files, and converting them into dataframe 

In [132]:
raw_tsv_files = glob.glob('*.tsv') # access all the files in the directory
base_df = pd.DataFrame() # create a dataframe to store all the values


for each_file in raw_tsv_files:
    total_data=[]
    with open(each_file, encoding='utf8', errors='ignore') as tsvfile:
        reader = csv.reader(tsvfile,delimiter='\t')
        for row in reader:
            total_data.append(row)
        
        df=pd.DataFrame(total_data)
        ############
        # Making the first row as dataframe headers
        df.columns = df.iloc[0]
        df = df[1:]
        ############
        base_df=pd.concat([base_df,df])

In [148]:
# replacing all space with nulls

base_df = base_df.replace(r'^\s*$', np.nan, regex=True)


In [159]:
main_df = base_df.copy()

In [160]:
main_df.isnull().sum()

0
host               0
logname            0
time               0
method             0
url                0
response           0
bytes              0
referer      3461595
useragent    3461595
dtype: int64

#### From above, we can observe that collumns = referer and useragent are nulls, hence selecting only required columns

In [161]:
main_df=main_df[['host', 'time', 'method', 'url', 'response', 'bytes']]
main_df['response']=main_df.response.astype('int64')
main_df['bytes']=main_df.bytes.astype('int64')

In [162]:
main_df['method'].unique().tolist()

['GET', 'HEAD', 'POST']

In [163]:
main_df['response'].unique().tolist()

['200', '304', '302', '404', '403', '500', '501']

---- 

## 2- Data wrangling

### 2.1) Filtering only valid URL pattern

The columns URL contains invalid URL, filtering them from the main data

In [141]:
print(main_df[~main_df.url.str.contains('[A-Za-z]')]['url'].unique().tolist())

['/', '/`', '/.', '/./', '/*.*', '//', '/1', '/+', '://', '/*', '/\x08\x08\x08']


In [142]:
main_df = main_df[main_df.url.str.contains('[A-Za-z]')]
main_df.shape

(3398303, 7)

### 2.2)  Converting seconds to date-time format

Since the original data is given in secons, converting it to date time format through a custom function

In [18]:
"""
Function - convert_time()
input - accept an integer
output - date time format
"""

def convert_time(date_str):
    date_str=int(date_str)
    new_date_str = (datetime.utcfromtimestamp(date_str) -  timedelta(hours=5, minutes=0)).strftime('%Y-%m-%d %H:%M:%S')
    return new_date_str

In [165]:
main_df['time']=main_df.swifter.apply(lambda x:convert_time(x['time']),axis=1)

In [16]:
main_df.head()

,host,time,method,url,response,bytes
1,199.72.81.55,1995-06-30 23:00:01,GET,/history/apollo/,200,6245
2,unicomp6.unicomp.net,1995-06-30 23:00:06,GET,/shuttle/countdown/,200,3985
3,199.120.110.21,1995-06-30 23:00:09,GET,/shuttle/missions/sts-73/mission-sts-73.html,200,4085
4,burger.letters.com,1995-06-30 23:00:11,GET,/shuttle/countdown/liftoff.html,304,0
5,199.120.110.21,1995-06-30 23:00:11,GET,/shuttle/missions/sts-73/sts-73-patch-small.gif,200,4179


### 2.3) From host_name/ IPaddress identifying the user's country


In [53]:
"""
Function - getip()
input - accept hostname / IP address
output - Country ISO code

Logic:
For successful result store the country ISOcode or if unsuccessful, store null
"""


def getip(domain_str):
    try:
        ip = socket.gethostbyname(domain_str.strip())
        reader = geolite2.reader()      
        output = reader.get(ip)
        result = output['country']['iso_code']
    except:
        result = np.NAN
    return result

In [5]:
# Approach on how to implement above Function

#Step-1: Since we are interested in the country information for unqiue visitors, filter them first

unique_host_names_list = main_df['host'].unique().tolist()
len(unique_host_names_list)

137978

In [ ]:
# Step-2: Create an empy dictionary to store each vistior and respective country

country_dic =dict()

for each_host in unique_host_names_list:
    country_dic.update({each_host:getip(each_host)})
    
# Step-3: Now map the results back to main dataframe    
main_df['country'] = main_df['host'].map(country_dic)

In [150]:
main_df['country'].unique()

array(['US', nan, 'AU', 'CA', 'KR', 'JP', 'CL', 'DE', 'CN', 'NZ', 'BR',
       'VG', 'RU', 'SE', 'IN', 'AT', 'GU', 'IT', 'JM', 'PL', 'MY', 'TW',
       'BE', 'FI', 'HK', 'TH', 'NL', 'FR', 'CH', 'GB', 'ZA', 'IL', 'DK',
       'NO', 'GR', 'MX', 'PT', 'ES', 'CR', 'AR', 'SG', 'SK', 'IS', 'LT',
       'SI', 'HN', 'HU', 'ID', 'LU', 'TR', 'IE', 'LV', 'PR', 'VE', 'UA',
       'KY', 'EC', 'IR', 'PE', 'UY', 'PH', 'CY', 'CO', 'HT', 'NG', 'AG',
       'TZ', 'TT', 'CZ', 'AW', 'RO', 'EG', 'FJ', 'SA', 'BM', 'EE', 'BB',
       'SO', 'CW', 'RE', 'BH', 'BY', 'AS', 'PY', 'HR', 'PK', 'KW', 'SY',
       'AE', 'BG', 'AQ', 'MU', 'MZ', 'BO', 'PA', 'SC'], dtype=object)

#### In total, visitors represent 96 countires ; city and GeoPoint in further stages can be created similarly

![](images/countries.PNG)

------

## 3 - Data Analysis & EDA

## HTTP Status Code Analysis

In [255]:
http_df = main_df.groupby(['response'])[['host']].count().sort_values(by='host',ascending=False)
http_df.reset_index(inplace=True)

In [256]:
http_df

,response,host
0,200,3043539
1,304,260478
2,302,73064
3,404,20893
4,403,225
5,500,65
6,501,39


![](images/http.PNG)

------

## Top 10 requested URLs 

##### Count of number of times URLs have been requested (This information will help company to find out most popular pages and how frequently they are accessed)

In [ ]:
freq_url = main_df.groupby(['url'])[['host']].count().sort_values(by='host',ascending=False).head(10)
freq_url.reset_index(inplace=True)

In [265]:
freq_url

,url,host
0,/images/NASA-logosmall.gif,208798
1,/images/KSC-logosmall.gif,164976
2,/images/MOSAIC-logosmall.gif,127916
3,/images/USA-logosmall.gif,127082
4,/images/WORLD-logosmall.gif,125933
5,/images/ksclogo-medium.gif,121580
6,/ksc.html,83919
7,/images/launch-logo.gif,76009
8,/history/apollo/images/apollo-logo1.gif,68898
9,/shuttle/countdown/,64740


## Top Ten Error Endpoints


##### This information is helpful for devops team to find out how many requests are failing so that appropriate action can be taken to fix the issue

In [270]:
error_endpoints_freq_df = main_df[main_df['response']!= 200]\
                .groupby(['url'])[['host']].count().sort_values(by='host',ascending=False).head(10)


error_endpoints_freq_df.reset_index(inplace=True)

In [271]:
error_endpoints_freq_df

,url,host
0,/images/NASA-logosmall.gif,40090
1,/images/KSC-logosmall.gif,23763
2,/images/MOSAIC-logosmall.gif,15245
3,/images/USA-logosmall.gif,15142
4,/images/WORLD-logosmall.gif,14773
5,/images/ksclogo-medium.gif,13559
6,/images/launch-logo.gif,8806
7,/history/apollo/images/apollo-logo1.gif,7489
8,/images/ksclogosmall.gif,5669
9,/shuttle/countdown/,5559


## Date analysis

In [520]:
main_df['dayofweek']=main_df['time'].dt.weekday+1
main_df['day']=main_df['time'].dt.day
main_df['month']=main_df['time'].dt.month
main_df['date']=main_df['time'].dt.date
main_df['log_time']=main_df['time'].dt.time

![](images/date.PNG)

There is peak traffic on 13th of July and the reason is space mission was lauched


![](images/mission.PNG)


---
### Weekly connections made

![](images/week.PNG)

##### From above we can observe that, weekedays the traffic is high but not on weekends

_______

## 4 - Creating new Features¶

### 4-1) Encoding categorical 'response'. Since majority of successful transcations are present, we can assign 0 and rest other unsuccessful to 1

In [524]:
response_encode = {200:0,304:1,302:1,404:1,403:1,500:1,501:1}

In [526]:
main_df['response'] = main_df['response'].map(response_encode)

### 4-2) Creating Country from hostname and IP address, based on requirement city and GeoCordinates can also be added

##### This step was completed in step- 2.3

### 4-3) From EDA, we observed that weekdays have high traffic and weekends have less, so we can encode these variables

* weekdays as 0
* weekends as 1

In [530]:
weekday_encode = {1:0,2:0,3:0,4:0,5:0,6:1,7:1}

In [531]:
main_df['is_weekday'] = main_df['dayofweek'].map(weekday_encode)
main_df.drop('dayofweek', axis=1, inplace=True)

### 4-4) Encoding HTTP method

* GET as 0
* POST as 1
* HEAD as 2

In [534]:
method_encode = {'GET':0,'POST':1,'HEAD':2}

In [535]:
main_df['method'] = main_df['method'].map(method_encode)


In [536]:
main_df.head()

,host,time,method,url,response,bytes,country,day,month,date,log_time,is_weekday
0,199.72.81.55,1995-06-30 23:00:01,0,/history/apollo/,0,6245,US,30,6,1995-06-30,23:00:01,0
1,unicomp6.unicomp.net,1995-06-30 23:00:06,0,/shuttle/countdown/,0,3985,NaN,30,6,1995-06-30,23:00:06,0
2,199.120.110.21,1995-06-30 23:00:09,0,/shuttle/missions/sts-73/mission-sts-73.html,0,4085,US,30,6,1995-06-30,23:00:09,0
3,burger.letters.com,1995-06-30 23:00:11,0,/shuttle/countdown/liftoff.html,1,0,NaN,30,6,1995-06-30,23:00:11,0
4,199.120.110.21,1995-06-30 23:00:11,0,/shuttle/missions/sts-73/sts-73-patch-small.gif,0,4179,US,30,6,1995-06-30,23:00:11,0


-----

## 4-5) Visitor Analysis

We will now examine the behavior of website visitors across time (sessions).

create a new Group recipe with visitor_id(ip address) as the column to group by. 

Output: 

*  average time in seconds spent by a customer in the website
*  average data used by same customer

In [566]:
def find_avg_values(df):
    
    visitor_df = df[['host','time','url','date','country','bytes']]
    grouped = visitor_df.groupby(['host','date'])
    another_df = pd.DataFrame()

    for x,y in grouped:
        if y.shape[0] >1:
            end_time = y['time'].max()
            start_time =y['time'].min()
            total_time = end_time - start_time
            y['time_spent'] = total_time.total_seconds()
            another_df=pd.concat([another_df,y])

        else:
            total_time = 30
            y['time_spent'] = total_time
            another_df=pd.concat([another_df,y])

    another_df = another_df[['host','date','time_spent','country','bytes']]
    another_df=another_df.drop_duplicates()


    visitor_avg_df = pd.DataFrame()

    second_group = another_df.groupby('host')

    for x,y in second_group:
        if y.shape[0]>1:
            y['avg_seconds']=y['time_spent'].mean()
            y['avg_data_used']=y['bytes'].mean()
            visitor_avg_df=pd.concat([visitor_avg_df,y])

        else:
            y['avg_seconds']=y['time_spent']
            y['avg_data_used']=y['bytes']
            visitor_avg_df=pd.concat([visitor_avg_df,y])

    visitor_avg_df = visitor_avg_df[['host','country','avg_seconds','avg_data_used']]
    visitor_avg_df=visitor_avg_df.drop_duplicates()
    visitor_avg_df = visitor_avg_df.sort_values(by=['avg_seconds','avg_data_used'], ascending=False)
    visitor_avg_df.reset_index(inplace=True,drop=True)
    
    
    return visitor_avg_df

In [568]:
sample_df=main_df.head(100000)

sample_df = find_avg_values(sample_df)
sample_df.shape

(7752, 4)

In [570]:
sample_df.head(10)

,host,country,avg_seconds,avg_data_used
0,compware.phx.net99.net,NaN,85144.000000,19653.692308
1,guest.dtc.net,NaN,84875.000000,67890.818182
2,ccfews03.center.osaka-u.ac.jp,NaN,81644.000000,23084.782609
3,netcom7.netcom.com,NaN,80829.000000,9923.800000
4,www-b5.proxy.aol.com,NaN,80504.148649,59324.918919
5,www-b1.proxy.aol.com,NaN,79823.103448,43117.525862
6,www-d1.proxy.aol.com,NaN,78474.630631,31888.247748
7,ix-ftl1-17.ix.netcom.com,NaN,78324.000000,21730.937500
8,fig.leba.net,PL,78312.000000,50024.111111
9,202.33.84.251,JP,75929.000000,9133.200000


In [571]:
main_df.head()

,host,time,method,url,response,bytes,country,day,month,date,log_time,is_weekday
0,199.72.81.55,1995-06-30 23:00:01,0,/history/apollo/,0,6245,US,30,6,1995-06-30,23:00:01,0
1,unicomp6.unicomp.net,1995-06-30 23:00:06,0,/shuttle/countdown/,0,3985,NaN,30,6,1995-06-30,23:00:06,0
2,199.120.110.21,1995-06-30 23:00:09,0,/shuttle/missions/sts-73/mission-sts-73.html,0,4085,US,30,6,1995-06-30,23:00:09,0
3,burger.letters.com,1995-06-30 23:00:11,0,/shuttle/countdown/liftoff.html,1,0,NaN,30,6,1995-06-30,23:00:11,0
4,199.120.110.21,1995-06-30 23:00:11,0,/shuttle/missions/sts-73/sts-73-patch-small.gif,0,4179,US,30,6,1995-06-30,23:00:11,0


# 5- Conclusison and recommendation

### Conclusion

Based on above dataframes, 

* we can further cluster web visitors into groups (frequent visitors, non-frequent visitors) in order to predict new customer behavior and optimise the website

* we can forecast how the web traffic can cause webpage breaks and overcome it


### Recommendation:

* The variables: Referrer and useragent can be recorded for analyse the trend on how visitors access the website
* Based on host-name; user's country, city and Geolocation can be accessed for further improving the model
